In [2]:
import sys
import os
sys.path = [os.path.abspath('../')] + sys.path

os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'

from experiments.plot_hawkes import plot_coeffs, plot_ticks_hist
from experiments.hawkes_coeffs import get_coeffs_dim_30, get_coeffs_dim_100, retrieve_coeffs
from experiments.simulation import simulate_and_save_hawkes, simulate_hawkes_in_parallel
from experiments.weights_computation import pre_compute_hawkes
from experiments.learning import find_best_metrics

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from multiprocessing import cpu_count

dim = 30
run_times = [5000, 7000, 10000, 15000, 20000]

n_simulations = 10
n_cpu = cpu_count() - 1

directory_path = '/home/m.bompaire/experiments/hawkes_data/'

n_decays = 3

In [3]:
# simulate_hawkes_in_parallel(dim, run_times, n_decays, n_simulations, directory_path, n_cpu=n_cpu)

decay_coeffs [[0.31806334 0.57734068 0.73284262 0.17472888 0.22129824 0.03905054]
 [0.06033171 0.70407832 0.36971643 0.7363282  0.98171675 0.57895883]
 [0.44699789 0.2664696  0.94853032 0.23742474 0.61177711 0.11477611]]
coeffs file existed already and was the same
/home/m.bompaire/experiments/hawkes_data/train_hawkes/dim_30/u=3/T_5000/simulation_010.npy
/home/m.bompaire/experiments/hawkes_data/train_hawkes/dim_30/u=3/T_7000/simulation_014.npy
/home/m.bompaire/experiments/hawkes_data/train_hawkes/dim_30/u=3/T_10000/simulation_013.npy
/home/m.bompaire/experiments/hawkes_data/train_hawkes/dim_30/u=3/T_15000/simulation_013.npy
/home/m.bompaire/experiments/hawkes_data/train_hawkes/dim_30/u=3/T_20000/simulation_013.npy
/home/m.bompaire/experiments/hawkes_data/train_hawkes/dim_30/u=3/T_5000/simulation_011.npy
/home/m.bompaire/experiments/hawkes_data/train_hawkes/dim_30/u=3/T_7000/simulation_011.npy
/home/m.bompaire/experiments/hawkes_data/train_hawkes/dim_30/u=3/T_10000/simulation_014.npy
/h

In [8]:
# max_pre_computed_hawkes = 100
# 
# for run_time in run_times:
#     pre_compute_hawkes(dim, run_time, n_decays, max_pre_computed_hawkes, 
#                        directory_path, n_cpu=n_cpu)

In [9]:
from collections import OrderedDict

from experiments.tested_prox import (
    create_prox_l1_no_mu, create_prox_l1w_no_mu,
    create_prox_l1_no_mu_nuclear, create_prox_l1w_no_mu_nuclear,
    create_prox_nuclear
)

prox_infos = OrderedDict()

prox_infos['l1'] = {
    'n_initial_points': 10,
    'max_relative_step': 1.4,
    'create_prox': create_prox_l1_no_mu,
    'tol': 1e-10,
    'dim': 1,
}

prox_infos['l1w'] = {
    'n_initial_points': 10,
    'max_relative_step': 1.4,
    'create_prox': create_prox_l1w_no_mu,
    'tol': 1e-10,
    'dim': 1,
}

prox_infos['nuclear'] = {
    'n_initial_points': 3,
    'max_relative_step': 5,
    'create_prox': create_prox_nuclear,
    'tol': 1e-7,
    'dim': 1,
}

prox_infos['l1_nuclear'] = {
    'n_initial_points': 3,
    'max_relative_step': 5,
    'create_prox': create_prox_l1_no_mu_nuclear,
    'tol': 1e-7,
    'dim': 2,
}

prox_infos['l1w_nuclear'] = {
    'n_initial_points': 3,
    'max_relative_step': 5,
    'create_prox': create_prox_l1w_no_mu_nuclear,
    'tol': 1e-7,
    'dim': 2,
}

prox_infos = OrderedDict(
    (prox_name, dict(name=prox_name, **prox_info)) for prox_name, prox_info in prox_infos.items())
list(prox_infos.keys())

['l1', 'l1w', 'nuclear', 'l1_nuclear', 'l1w_nuclear']

In [ ]:
n_models = 100

solver_kwargs = {'tol': 1e-6, 'max_iter': 25000}

for run_time in run_times:
    for prox_info in prox_infos.values():
        if run_time == 5000 and prox_info['name'] != 'l1w_nuclear':
            continue
            
        infos = find_best_metrics(
            dim, run_time, n_decays, n_models, prox_info, solver_kwargs,
            directory_path, max_run_count=10, n_cpu=n_cpu, suffix='100_x_25000_iter')


### For prox l1w_nuclear
decay_coeffs [[0.31806334 0.57734068 0.73284262 0.17472888 0.22129824 0.03905054]
 [0.06033171 0.70407832 0.36971643 0.7363282  0.98171675 0.57895883]
 [0.44699789 0.2664696  0.94853032 0.23742474 0.61177711 0.11477611]]
coeffs file existed already and was the same
Retrieved 100 precomputed models
2019-12-28 15:31 Run 0 - With 9 new points
needed 1448.22 seconds
2019-12-28 15:55 Run 1 - With 6 new points
needed 586.43 seconds
2019-12-28 16:05 Run 2 - With 13 new points


In [10]:
run_times

[5000, 7000, 10000, 15000, 20000]

In [ ]:
n_simulations = 100

simulate_hawkes_in_parallel(dim, run_times, n_decays, n_simulations, directory_path, n_cpu=n_cpu)

max_pre_computed_hawkes = 100

for run_time in run_times:
    pre_compute_hawkes(dim, run_time, n_decays, max_pre_computed_hawkes, 
                       directory_path, n_cpu=n_cpu)

n_models = 100

solver_kwargs = {'tol': 1e-6, 'max_iter': 25000}

for run_time in run_times:
    for prox_info in prox_infos.values():
        infos = find_best_metrics(
            dim, run_time, n_decays, n_models, prox_info, solver_kwargs,
            directory_path, max_run_count=10, n_cpu=n_cpu, suffix='100_x_25000_iter')


decay_coeffs [[0.31806334 0.57734068 0.73284262 0.17472888 0.22129824 0.03905054]
 [0.06033171 0.70407832 0.36971643 0.7363282  0.98171675 0.57895883]
 [0.44699789 0.2664696  0.94853032 0.23742474 0.61177711 0.11477611]]
coeffs file existed already and was the same
/home/m.bompaire/experiments/hawkes_data/train_hawkes/dim_30/u=3/T_5000/simulation_060.npy
/home/m.bompaire/experiments/hawkes_data/train_hawkes/dim_30/u=3/T_7000/simulation_061.npy
/home/m.bompaire/experiments/hawkes_data/train_hawkes/dim_30/u=3/T_10000/simulation_060.npy
/home/m.bompaire/experiments/hawkes_data/train_hawkes/dim_30/u=3/T_15000/simulation_060.npy
/home/m.bompaire/experiments/hawkes_data/train_hawkes/dim_30/u=3/T_20000/simulation_030.npy
/home/m.bompaire/experiments/hawkes_data/train_hawkes/dim_30/u=3/T_5000/simulation_063.npy
/home/m.bompaire/experiments/hawkes_data/train_hawkes/dim_30/u=3/T_7000/simulation_065.npy
/home/m.bompaire/experiments/hawkes_data/train_hawkes/dim_30/u=3/T_10000/simulation_071.npy
/h

In [7]:
from experiments.metrics import compute_metrics
from experiments.learning import learn_one_model
from experiments.hawkes_coeffs import retrieve_coeffs
from tick.solver import GFB
import numpy as np

model_file_name = '/home/m.bompaire/experiments/hawkes_data/train_hawkes/dim_30/u=3/T_5000/precomputed/precomputed_000.pkl'
strength_range = [(1e-08, 1e-08), (1e-08, 1e-05)]#, (1e-08, 0.01), (1e-05, 1e-08), (1e-05, 1e-05), (1e-05, 0.01), (0.01, 1e-08), (0.01, 1e-05), (0.01, 0.01)]
create_prox = create_prox_nuclear
original_coeffs = np.hstack((mu0, A0.reshape(dim * dim * n_decays)))#retrieve_coeffs(dim, n_decays, directory_prefix=directory_path)
SolverClass =  GFB
solver_kwargs = {'tol': 1e-07, 'max_iter': 1}
save_coeffs = False



In [8]:
%%prun 


learn_one_model(model_file_name, strength_range, create_prox,
                    compute_metrics, original_coeffs, SolverClass,
                    solver_kwargs, save_coeffs=save_coeffs)

worked with 30/30 (ie. full)
worked with 30/30 (ie. full)
2019-12-27 14:41 - did not converge train=0 strength=1e-08, 1e-08, stopped at 0.502 for tol=1e-07, took 0.018s
worked with 30/30 (ie. full)
worked with 30/30 (ie. full)
2019-12-27 14:41 - did not converge train=0 strength=1e-08, 1e-05, stopped at 0.502 for tol=1e-07, took 0.017s
 

         17793 function calls (17290 primitive calls) in 0.229 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.167    0.167    0.167    0.167 {built-in method tick.hawkes.model.build._hawkes_model.ModelHawkesSumExpKernLeastSqDeserialize}
        8    0.013    0.002    0.013    0.002 {built-in method tick.hawkes.model.build._hawkes_model.Model_loss}
        1    0.008    0.008    0.175    0.175 {built-in method _pickle.load}
        2    0.004    0.002    0.004    0.002 {built-in method tick.hawkes.model.build._hawkes_model.Model_grad}
      160    0.004    0.000    0.007    0.000 decomp_lu.py:152(lu)
      192    0.003    0.000    0.003    0.000 extmath.py:118(safe_sparse_dot)
       16    0.003    0.000    0.003    0.000 decomp_svd.py:16(svd)
      340    0.002    0.000    0.002    0.000 {method 'reduce' of 'numpy.ufunc' objects}
       12    0.001    0.000    0.001    0.000 {method 'argsort' of 'numpy.ndarr

In [26]:
import pickle
import numpy as np
precomputed_model = pickle.load(open(model_file_name, 'rb'))

In [6]:
%%prun

precomputed_model._model.loss(np.hstack((mu0, A0.reshape(dim * dim * n_decays))))

NameError: name 'precomputed_model' is not defined

In [5]:
betas, mu0, A0 = retrieve_coeffs(dim, n_decays, directory_path)

decay_coeffs [[0.31806334 0.57734068 0.73284262 0.17472888 0.22129824 0.03905054]
 [0.06033171 0.70407832 0.36971643 0.7363282  0.98171675 0.57895883]
 [0.44699789 0.2664696  0.94853032 0.23742474 0.61177711 0.11477611]]
coeffs file existed already and was the same


In [29]:
%%time

from sklearn.utils.extmath import randomized_svd
M = A0.reshape(dim, -1)
_, s, _ = randomized_svd(M, min(M.shape), n_iter=5)
s

CPU times: user 93.7 ms, sys: 532 ms, total: 626 ms
Wall time: 21.9 ms


array([4.78113239e-01, 1.81247761e-01, 1.42928894e-01, 1.12299968e-01,
       6.80453646e-02, 5.98226417e-02, 4.24221689e-17, 3.84804351e-17,
       3.84804351e-17, 3.84804351e-17, 3.84804351e-17, 3.84804351e-17,
       3.84804351e-17, 3.84804351e-17, 3.84804351e-17, 3.84804351e-17,
       3.84804351e-17, 3.84804351e-17, 3.84804351e-17, 3.84804351e-17,
       3.84804351e-17, 3.84804351e-17, 3.84804351e-17, 3.84804351e-17,
       3.84804351e-17, 3.84804351e-17, 3.84804351e-17, 3.84804351e-17,
       3.84804351e-17, 4.44619827e-18])

In [59]:
%%time

from skcuda import linalg
skcuda.misc.init()

M = A0.reshape(dim, -1)
import pycuda.gpuarray as gpuarray
M_gpu = gpuarray.to_gpu(M)
s = linalg.svd(M_gpu, jobu='N', jobvt='N')
s

CPU times: user 5.88 ms, sys: 3.66 ms, total: 9.54 ms
Wall time: 8.96 ms


array([4.78113239e-001, 1.81247761e-001, 1.42928894e-001, 1.12299968e-001,
       6.80453646e-002, 5.98226417e-002, 4.01404016e-017, 3.26034948e-017,
       2.60687461e-017, 1.49185513e-017, 1.12065609e-017, 9.67732282e-018,
       4.31306908e-018, 9.17757117e-033, 2.63143077e-033, 1.67531010e-033,
       1.22436643e-033, 5.34249724e-049, 2.23998672e-049, 9.44582706e-051,
       1.11824217e-064, 1.50342853e-065, 1.51961767e-066, 1.09009454e-080,
       1.91514972e-083, 6.07217095e-097, 6.00108353e-100, 3.54081513e-113,
       3.17045794e-130, 7.98805919e-147])

In [64]:
skcuda.misc.sum(s)

array(1.04245787)

In [47]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [ ]:
%%time

import os
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'

import scipy
import scipy.linalg
import numpy as np
n = 10000
_, s, _ = scipy.linalg.svd(np.random.rand(n, n))


In [72]:
np.__config__.show()

blas_mkl_info:
  NOT AVAILABLE
blis_info:
  NOT AVAILABLE
openblas_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
blas_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
lapack_mkl_info:
  NOT AVAILABLE
openblas_lapack_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
lapack_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
